In [54]:
import pandas as pd
import seaborn as sns

In [55]:
customers = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-fahretdinov/E-commerce/datasets/olist_customers_dataset.csv')
order_items = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-fahretdinov/E-commerce/datasets/olist_order_items_dataset.csv')
orders = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-fahretdinov/E-commerce/datasets/olist_orders_dataset.csv')

In [56]:
# №0. -Что считается покупкой?
delivered_orders = orders.query("order_status == 'delivered'")
delivered_orders.order_status.count()
# -Покупкой считается доставленный товар (order_status == delivered)

96478

In [57]:
# №1. Сколько у нас пользователей, которые совершили покупку только один раз? 

# делаем merge c таблицами customers и delivered_orders по customer_id (т.к.customer_id идентичны в обоих таблицах), 
#чтобы выявить клиентов, которые совершили покупку ("order_status == 'delivered'"):

df = pd.merge(customers,delivered_orders, on='customer_id')

# т.к. customer_unique_id -уникальный номер клиента, а customer_id присваивается клиенту на каждый новый заказ,
# то можем посчитать кол-во customer_id у customer_unique_id:

one_order_clients = df.groupby('customer_unique_id', as_index=False) \
    .agg({'customer_id': 'count'}) \
    .query("customer_id == 1").count() # -отфильтровали тех, кто сделал только одну покупку
one_order_clients.customer_id
# 90557 - столько клиентов сделали 1 покупку

90557

In [58]:
#2 Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [59]:
# 1) перевести нужные колонки с датами в нужный формат
# 2) переформатировать даты с месяца (где 1-январь, 2-февраль и т.д.)
# 3) сгруппировать по месяцам и статусам заказа
# 4) посчитать кол-во order_id (т.к. order_id- уникальный идентификатор заказа) и вывести детализацию по 
# месяцам и статусам заказаов (нас итнересуют canceled and unavailable)
# 5) вывести среднее (mean) по недоставленным заказам по месяцам

In [60]:
orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [61]:
# конвертируем в дату колонки с датами:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])

In [62]:
# вопрос для разбора- как конвертироать в date те данные, в столбцах которых содержится date?
#for i in orders:
#    if i has 'date':
#       apply(pd.to_datetime(orders[i]))

In [63]:
# будем работать по order_purchase_timestamp -время создания заказа
# извлекаем месяц из колонки:
orders['order_purchase_timestamp'] = orders['order_purchase_timestamp'].dt.month

In [64]:
abc = orders.groupby(['order_purchase_timestamp', 'order_status'], as_index=False) \
    .agg({'order_id': 'count'}).query("order_status == 'canceled' or order_status== 'unavailable'")
#сгруппировать по месяцам и статусам заказа
# детализация недоставленных заказов по месяцам
# недоставленным заказом будем считать "order_status == 'canceled' or order_status== 'unavailable" 
abc

,order_purchase_timestamp,order_status,order_id
0,1,canceled,37
5,1,unavailable,58
7,2,canceled,90
13,2,unavailable,75
14,3,canceled,59
19,3,unavailable,49
21,4,canceled,33
26,4,unavailable,14
27,5,canceled,53
32,5,unavailable,47


In [65]:
# считаем среднее кол-во недоставленных заказов, убирая детализацию по статусу заказа (в датафрейме "abc"
# хранятся лишь "order_status == 'canceled' or order_status== 'unavailable)
abc.rename(columns={'order_id': 'orders'}).groupby('order_purchase_timestamp') \
    .agg({'orders': 'sum'}).mean()
# 102.833333 заказа в  месяц в среднем не доставляется вовремя

orders    102.833333
dtype: float64

In [66]:
# 3. По каждому ТОВАРУ определить, в какой день недели товар чаще всего покупается.

In [67]:
# a)product_id - ид товара. мерджим с delivered_orders, т.к. нас спрашивают именно про каждый товар, а не заказ 
# (в заказе, к примеру, может быть несколько разных товаров);
# b)order_delivered_customer_date переводим в дни недели (где 0 -Monday и т.п.);
# c)группируем новый дф по product_id и order_delivered_customer_date ( дата доставки);
# и считаем order_item_id (т.к. order_item_id - показывает,сколько товаров было в заказе).

In [68]:
#a)
task_3 = pd.merge(order_items,delivered_orders, on='order_id')
task_3.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [69]:
#b)
task_3['order_delivered_customer_date'] = pd.to_datetime(task_3['order_delivered_customer_date'])
task_3['order_delivered_customer_date'] = task_3['order_delivered_customer_date'].dt.dayofweek

In [70]:
#c)
task_3.groupby(['product_id', 'order_delivered_customer_date'],as_index=False) \
    .agg({'order_item_id':'sum'}).sort_values('order_item_id', ascending=False)

,product_id,order_delivered_customer_date,order_item_id
15029,422879e10f46682990de24d770e7f83d,3.0,232
53620,ee3d532c8a438679776d222e997606b3,0.0,211
33522,9571759451b1d780ee7c15012ea109d4,5.0,210
15026,422879e10f46682990de24d770e7f83d,0.0,165
37314,a62e25e09e05e6faf31d90c6ec1aa3d1,0.0,163
...,...,...,...
23227,66740f6bc8c6a2393283871b22d5d32f,0.0,1
23228,66740f6bc8c6a2393283871b22d5d32f,2.0,1
23229,66740f6bc8c6a2393283871b22d5d32f,4.0,1
23230,667b61f3396804596ee56677af687d3c,4.0,1


In [71]:
# как итог, мы можем с помощью .query выбрать нужный нам product_id и посмотреть его продажи в разные дни недели:
task_3.groupby(['product_id', 'order_delivered_customer_date'],as_index=False) \
    .agg({'order_item_id':'count'}).sort_values('order_delivered_customer_date', ascending=True) \
    .query("product_id == '422879e10f46682990de24d770e7f83d'")

,product_id,order_delivered_customer_date,order_item_id
15026,422879e10f46682990de24d770e7f83d,0.0,102
15027,422879e10f46682990de24d770e7f83d,1.0,86
15028,422879e10f46682990de24d770e7f83d,2.0,101
15029,422879e10f46682990de24d770e7f83d,3.0,104
15030,422879e10f46682990de24d770e7f83d,4.0,57
15031,422879e10f46682990de24d770e7f83d,5.0,32
15032,422879e10f46682990de24d770e7f83d,6.0,2


In [72]:
#5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту
# с самым высоким retention на 3й месяц

In [73]:
# делаем merge c таблицами customers и delivered_orders по customer_id (т.к.customer_id идентичны в обоих таблицах), 
#чтобы выявить клиентов, которые совершили покупку ("order_status == 'delivered'")
# чтобы узнать, есть ли клиенты, которые сделали >= 1 покупки:
df_5 = pd.merge(customers,delivered_orders, on='customer_id')

In [74]:
# так как дата покупки в датафрейме за 2 года(2017 и 2018 года), я возьму когорту за 2017

# для начала переведем колонку в нужный формат:
df_5['order_delivered_customer_date'] = pd.to_datetime(df_5['order_delivered_customer_date'])

# далее отфильтруем дф только по 2017 году и перезапишем датасет:
df_5 = df_5.query("order_delivered_customer_date < '01-01-2018'")


In [75]:
# создаю отдельную колонку с месяцем покупки:
df_5['month_of_buy'] = df_5['order_delivered_customer_date'].dt.month

# в отдельную таблицу взяли столбцы уникального айди и месяц-год, отфортировали по месяц-год и удалили дубли
# (удалили дубли, так как нам не нужна информация о том, что, к примеру, один пользователь сделал несколько покупок
# в один месяц)
df_new = df_5[['customer_unique_id', 'month_of_buy']].copy()
df_new = df_new.drop_duplicates().sort_values('month_of_buy', ascending=False)

In [76]:
# сгруппировали по уникальному айди, в агрегации нарастающий счет по месяцу
# (где 0 - это месяц первой покупки, то есть когорта)
df_new['cohort'] = df_new.groupby('customer_unique_id').agg({'month_of_buy':'cumcount'})

In [88]:
# снова группируем датафрейм, чтобы посчитать кол-во покупок клиентов по месяцам:
df_cohort = df_new.groupby(['month_of_buy', 'cohort'],as_index=False).agg({'customer_unique_id': 'count'})
df_cohort.head(3)

,month_of_buy,cohort,customer_unique_id
0,1,0,253
1,1,1,10
2,1,2,1


In [89]:
# переформатируем датафрейм, где индекс- месяц года, колонки- когорта(месяц) повторных покупок

In [90]:
df_coh = df_cohort.pivot(index='month_of_buy', columns='cohort', values='customer_unique_id')

In [91]:
df_coh

cohort,0,1,2,3,4,5
month_of_buy,,,,,,
1,253.0,10.0,1.0,NaN,NaN,NaN
2,1299.0,27.0,2.0,NaN,NaN,NaN
3,2293.0,58.0,3.0,NaN,NaN,NaN
4,1776.0,44.0,1.0,NaN,NaN,NaN
5,3606.0,89.0,4.0,NaN,NaN,NaN
6,3093.0,71.0,6.0,NaN,NaN,1.0
7,3343.0,56.0,2.0,1.0,1.0,NaN
8,4151.0,68.0,3.0,1.0,NaN,NaN
9,3838.0,64.0,3.0,1.0,NaN,NaN


In [92]:
# делим столбцы когорт , начиная с первой, на нулевую когорту, чтобы из абсолютных величин перейти в относительные:
list_number = df_coh.iloc[:,0]
lst = df_coh.columns

df_coh_coef = df_coh[lst] = df_coh[lst].apply(lambda c: (c/list_number))

In [93]:
df_coh_coef

cohort,0,1,2,3,4,5
month_of_buy,,,,,,
1,1.0,0.039526,0.003953,NaN,NaN,NaN
2,1.0,0.020785,0.001540,NaN,NaN,NaN
3,1.0,0.025294,0.001308,NaN,NaN,NaN
4,1.0,0.024775,0.000563,NaN,NaN,NaN
5,1.0,0.024681,0.001109,NaN,NaN,NaN
6,1.0,0.022955,0.001940,NaN,NaN,0.000323
7,1.0,0.016751,0.000598,0.000299,0.000299,NaN
8,1.0,0.016382,0.000723,0.000241,NaN,NaN
9,1.0,0.016675,0.000782,0.000261,NaN,NaN


In [94]:
# тк первый месяц ретеншена- это колонка 0, то 3 месяцем будет колонка 2: самый высокий ретеншен в этой колонке- 
# 0.001940. Можно сделать вывод, что в первые месяца исследования приходит много пользователей, однако уровень
# их удержания очень низкий.